In [3]:
import requests

def get_latest_valid_price(data_array):
    """Keepaの価格配列から最新の有効な新品価格を取得"""
    if not isinstance(data_array, list):
        return None
    for i in range(len(data_array) - 1, -1, -2):
        price = data_array[i]
        if isinstance(price, (int, float)) and price > 0:
            return price  # 日本は円単位
    return None


def fetch_keepa_new_price(api_key, jan):
    """JANコードを指定して、新品価格を取得（履歴 or 出品）"""
    url = f"https://api.keepa.com/product?key={api_key}&domain=5&code={jan}&history=1&offers=20"
    response = requests.get(url)
    data = response.json()

    if "products" in data and len(data["products"]) > 0:
        product = data["products"][0]

        # タイトル
        title = product.get("title", "タイトルなし")

        # まず新品価格履歴（csv[1]）をチェック
        csv = product.get("csv", [])
        new_price_history = csv[1] if len(csv) > 1 else []
        latest_new_price = get_latest_valid_price(new_price_history)

        if latest_new_price:
            return title, f"{latest_new_price} 円（新品）"

        # 新品履歴が無ければ offers から探す
        offers = product.get("offers", [])
        min_new_offer = None
        for offer in offers:
            if offer.get("isNew", False):  # 新品のみ
                price = offer.get("price", -1)
                if price > 0:
                    if min_new_offer is None or price < min_new_offer:
                        min_new_offer = price

        if min_new_offer:
            return title, f"{min_new_offer} 円（新品）"

        return title, None
    else:
        return None, None


if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "194735186594"  # ← JANコード

    title, price = fetch_keepa_new_price(API_KEY, JAN)

    if title and price:
        print(f"商品タイトル: {title}")
        print(f"新品価格: {price}")
    else:
        print("新品が見つかりませんでした。")

商品タイトル: ホットウィール(Hot Wheels) カーカルチャー ジャパンヒストリックス4 日産フェアレディZ 乗り物おもちゃ ミニカー 3歳から ネイビー HRV86
新品価格: 1100 円（新品）


In [14]:
import requests
import time

def get_latest_valid_price(data_array):
    """Keepaの価格配列から最新の有効な新品価格を取得"""
    if not isinstance(data_array, list):
        return None
    for i in range(len(data_array) - 1, -1, -2):  # 後ろから2ステップずつ（時刻・価格ペア）
        price = data_array[i]
        if isinstance(price, (int, float)) and price > 0:
            return price  # 日本は円単位
    return None


def fetch_keepa_new_price(api_key, jan, max_retries=3):
    """JANコードを指定して、新品価格を取得（複数ヒット対応＋最安値抽出＋再試行対応）"""
    url = f"https://api.keepa.com/product?key={api_key}&domain=5&code={jan}&history=1&offers=20"

    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=30)  # ← タイムアウト30秒に拡張
            data = response.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト発生（{attempt+1}/{max_retries}回目）: {jan}")
            time.sleep(3)  # 少し待って再試行
        except Exception as e:
            print(f"⚠️ 予期せぬエラー: {e}")
            return None, None
    else:
        print(f"❌ 最大リトライ回数を超えました: {jan}")
        return None, None

    # 商品が存在しない場合
    if "products" not in data or len(data["products"]) == 0:
        return None, None

    lowest_price = None
    lowest_title = None

    # 🔁 複数ヒットに対応
    for product in data["products"]:
        title = product.get("title", "タイトルなし")

        # ① csv履歴から新品価格を取得
        csv = product.get("csv", [])
        new_price_history = csv[1] if len(csv) > 1 else []
        latest_new_price = get_latest_valid_price(new_price_history)

        # ② 履歴がなければ offers から探す
        if not latest_new_price:
            offers = product.get("offers", [])
            min_new_offer = None
            for offer in offers:
                if offer.get("isNew", False):
                    price = offer.get("price", -1)
                    if price > 0:
                        if min_new_offer is None or price < min_new_offer:
                            min_new_offer = price
            if min_new_offer:
                latest_new_price = min_new_offer

        # ③ 最安値更新
        if latest_new_price:
            if lowest_price is None or latest_new_price < lowest_price:
                lowest_price = latest_new_price
                lowest_title = title

    if lowest_price:
        return lowest_title, f"{lowest_price} 円（最安新品）"

    return None, None


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "9784813523017"  # ← JANコード

    title, price = fetch_keepa_new_price(API_KEY, JAN)

    if title and price:
        print(f"商品タイトル: {title}")
        print(f"新品価格: {price}")
    else:
        print("新品価格が見つかりませんでした。")


商品タイトル: シャークアタック&Co. ([バラエティ])
新品価格: 548 円（最安新品）


In [64]:
# うまくいったが、送料考えない。
import requests
import time

def get_latest_valid_price(data_array):
    """Keepaの価格配列から最新の有効な新品価格を取得"""
    if not isinstance(data_array, list):
        return None
    for i in range(len(data_array) - 1, -1, -2):
        price = data_array[i]
        if isinstance(price, (int, float)) and price > 0:
            return price
    return None


def fetch_keepa_lowest_new_price(api_key, jan, max_retries=3):
    """新品履歴を優先し、なければoffersから補完して最安新品を取得"""
    url = f"https://api.keepa.com/product?key={api_key}&domain=5&code={jan}&history=1&offers=20"

    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=30)
            data = response.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {jan}")
            time.sleep(3)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None
    else:
        print(f"❌ 最大リトライ回数を超えました: {jan}")
        return None, None

    if "products" not in data or len(data["products"]) == 0:
        return None, None

    lowest_price = None
    lowest_title = None

    # 複数ヒット全件をチェック
    for product in data["products"]:
        title = product.get("title", "タイトルなし")

        # 1️⃣ 履歴から新品価格を取得
        csv = product.get("csv", [])
        new_price_history = csv[1] if len(csv) > 1 else []
        latest_new_price = get_latest_valid_price(new_price_history)

        # 2️⃣ 履歴がない場合、offersから補完
        if not latest_new_price:
            offers = product.get("offers", [])
            offer_prices = [
                o.get("price") for o in offers
                if o.get("isNew", False)
                and isinstance(o.get("price"), (int, float))
                and o.get("price") > 0
            ]
            if offer_prices:
                latest_new_price = min(offer_prices)

        # 3️⃣ 最安値更新
        if latest_new_price:
            if lowest_price is None or latest_new_price < lowest_price:
                lowest_price = latest_new_price
                lowest_title = title

    if lowest_price:
        return lowest_title, f"{lowest_price} 円（最安新品）"
    else:
        return None, None


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4901001283927"  # ← 海底撈 即席火鍋マーラービーフ

    title, price = fetch_keepa_lowest_new_price(API_KEY, JAN)

    if title and price:
        print(f"商品タイトル: {title}")
        print(f"新品価格: {price}")
    else:
        print("新品価格が見つかりませんでした。")


商品タイトル: 味の素 50g
新品価格: 444 円（最安新品）


In [84]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp

def fetch_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int]]:
    """
    商品ページを開いた直後に表示される価格を取得する。
    1) Buy Box（最優先）
    2) isPrime == True の出品（ページ表示順で最初）
    3) 上記がなければ liveOffersOrder の先頭
    戻り値: (title, price)  ※priceは円（整数）
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"          # 履歴なし
        "&offers=10"          # 出品10件まで（軽い）
        "&onlyLiveOffers=1"   # 現在の出品のみ
        "&stats=90"           # BuyBox含む統計
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(3)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした。")
        return None, None

    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ① Buy Box（ページトップの価格）
    stats = product.get("stats") or {}
    for k in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(k)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v)

    # ② Prime優先 → ③ 先頭offer
    offers = product.get("offers", []) or []
    order = product.get("liveOffersOrder", []) or []
    ordered_offers = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered_offers and offers:
        ordered_offers = offers

    prime_offer = next((o for o in ordered_offers if o.get("isPrime")), None)
    chosen = prime_offer or (ordered_offers[0] if ordered_offers else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            price = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, price

    return None, None


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4582526449151"  # 例：商品JANコード

    title, price = fetch_display_price(API_KEY, JAN)

    print("────────────────────────────")
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格（BuyBox想定）: {price} 円")
    else:
        print("❌ 価格が取得できませんでした。")
    print("────────────────────────────")


❌ 商品情報が見つかりませんでした。
────────────────────────────
❌ 価格が取得できませんでした。
────────────────────────────


In [98]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp

def fetch_top_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int]]:
    """
    ページトップに表示される価格（BuyBox）を最優先で取得。
    BuyBoxがなければ Prime か最初の出品を取得。
    戻り値: (title, price)  ※priceは円（整数）
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"          # 履歴はいらない（軽量化）
        "&offers=20"          # 出品情報を20件まで取る
        "&onlyLiveOffers=1"   # 現在出品のみ
        "&stats=90"           # BuyBox統計情報を含める
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        return None, None

    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ————— ① BuyBox（トップ表示価格）を最優先 —————
    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v)

    # ————— ② フォールバック：Prime 出品または最初の出品 —————
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total

    return None, None


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "194735186594"  # 例：商品JAN

    title, price = fetch_top_display_price(API_KEY, JAN)
    print("────────────────────────────")
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("❌ 価格取得失敗")
    print("────────────────────────────")


────────────────────────────
商品タイトル: ホットウィール(Hot Wheels) カーカルチャー ジャパンヒストリックス4 日産フェアレディZ 乗り物おもちゃ ミニカー 3歳から ネイビー HRV86
ページトップ表示価格: 1100 円
────────────────────────────


In [101]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp

def fetch_top_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int], int]:
    """
    ページトップに表示される価格（BuyBox）を最優先で取得。
    BuyBoxがなければ Prime か最初の出品を取得。
    戻り値: (title, price, hit_count)
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"          # 履歴なしで軽量化
        "&offers=20"          # 出品情報を20件まで取得
        "&onlyLiveOffers=1"   # 現在出品のみ
        "&stats=90"           # BuyBox統計を含む
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        return None, None, 0

    hit_count = len(data["products"])
    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ————— ① BuyBox（トップ表示価格）を最優先 —————
    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count

    # ————— ② フォールバック：Prime 出品または最初の出品 —————
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count

    return None, None, hit_count


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "9784813523017"  # 例：商品JAN

    title, price, hit_count = fetch_top_display_price(API_KEY, JAN)

    print("────────────────────────────")
    print(f"🔍 ヒット件数: {hit_count} 件")
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("❌ 価格取得失敗")
    print("────────────────────────────")


⚠️ エラー: 429 Client Error: Too Many Requests for url: https://api.keepa.com/product?key=5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j&domain=5&code=9784813523017&history=0&offers=20&onlyLiveOffers=1&stats=90
────────────────────────────
🔍 ヒット件数: 0 件
❌ 価格取得失敗
────────────────────────────


In [115]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp

def fetch_top_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int], int, int, int]:
    """
    ページトップに表示される価格（BuyBox）を最優先で取得。
    BuyBoxがなければ Prime か最初の出品を取得。
    戻り値: (title, price, hit_count, tokens_consumed, tokens_left)
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"          # 履歴なしで軽量化
        "&offers=20"          # 出品情報を20件まで取得
        "&onlyLiveOffers=1"   # 現在出品のみ
        "&stats=0"           # BuyBox統計を含める
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0, 0, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        return None, None, 0, data.get("tokensConsumed", 0), data.get("tokensLeft", 0) if data else (0, 0)

    hit_count = len(data["products"])
    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)

    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ————— ① BuyBox（トップ表示価格）を最優先 —————
    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count, tokens_consumed, tokens_left

    # ————— ② フォールバック：Prime 出品または最初の出品 —————
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count, tokens_consumed, tokens_left

    return None, None, hit_count, tokens_consumed, tokens_left


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4582526449151"  # 例：商品JAN

    title, price, hit_count, tokens_used, tokens_left = fetch_top_display_price(API_KEY, JAN)

    print("────────────────────────────")
    print(f"🔍 ヒット件数: {hit_count} 件")
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("❌ 価格取得失敗")
    print("────────────────────────────")
    print(f"🪙 消費トークン: {tokens_used}")
    print(f"💰 残トークン数: {tokens_left}")
    print("────────────────────────────")


"""

トークン消費多すぎるやつ

"""


────────────────────────────
🔍 ヒット件数: 1 件
商品タイトル: KR 「ホルダーネック」 穴あきボディストッキング(KRW15)
ページトップ表示価格: 900 円
────────────────────────────
🪙 消費トークン: 1
💰 残トークン数: 37
────────────────────────────


'\n\nトークン消費多すぎるやつ\n\n'

In [113]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp

def fetch_top_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int], int, int, int]:
    """
    ページトップに表示される価格を取得。
    ヒットが2件以上ならスキップ。
    戻り値: (title, price, hit_count, tokens_consumed, tokens_left)
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"
        "&offers=5"           # 出品を5件まで取得（軽量かつ精度◎）
        # "&onlyLiveOffers=1"  # ← 一時的に外す（出品がない場合の保険）
        "&stats=90"           # BuyBox情報を含める
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0, 0, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        return None, None, 0, 0, 0

    hit_count = len(data["products"])
    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)

    if hit_count > 1:
        print(f"⚠️ ヒット多数（{hit_count}件）: 判別不可 → pass")
        return None, None, hit_count, tokens_consumed, tokens_left

    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ① BuyBoxから
    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count, tokens_consumed, tokens_left

    # ② offersから
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    if ordered:
        chosen = ordered[0]
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count, tokens_consumed, tokens_left

    return title, None, hit_count, tokens_consumed, tokens_left


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4582526449151"  # テスト用

    title, price, hit_count, tokens_used, tokens_left = fetch_top_display_price(API_KEY, JAN)

    print("────────────────────────────")
    print(f"🔍 ヒット件数: {hit_count} 件")
    if hit_count > 1:
        print("⚠️ 判別不可（ヒット多数のためスキップ）")
    elif title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print(f"❌ 価格が取得できませんでした（title: {title}）")
    print("────────────────────────────")
    print(f"🪙 消費トークン: {tokens_used}")
    print(f"💰 残トークン数: {tokens_left}")
    print("────────────────────────────")


❌ 商品情報が見つかりませんでした
────────────────────────────
🔍 ヒット件数: 0 件
❌ 価格が取得できませんでした（title: None）
────────────────────────────
🪙 消費トークン: 0
💰 残トークン数: 0
────────────────────────────


In [112]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp

def fetch_top_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int], int, int, int]:
    """
    ページトップに表示される価格（BuyBox）を最優先で取得。
    トークン消費が3を超えた場合は「トークンオーバー」として処理中止。
    戻り値: (title, price, hit_count, tokens_consumed, tokens_left)
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"         # 履歴なしで軽量化
        "&offers=5"          # 出品を5件まで取得
        "&onlyLiveOffers=1"  # 現在出品のみ
        "&stats=0"           # 統計データを省略
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0, 0, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        return None, None, 0, 0, 0

    # トークン消費量を取得
    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)

    # 🧠 トークン監視（3超えたら中止）
    if tokens_consumed > 3:
        print(f"⚠️ トークンオーバー：{tokens_consumed} トークン消費。処理中止。")
        return None, None, 0, tokens_consumed, tokens_left

    hit_count = len(data["products"])
    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ① BuyBox（statsなしなので多くはNoneになるが念のため）
    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count, tokens_consumed, tokens_left

    # ② Prime優先 or 最初の出品
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count, tokens_consumed, tokens_left

    return None, None, hit_count, tokens_consumed, tokens_left


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4582526449151"  # テスト用JAN

    title, price, hit_count, tokens_used, tokens_left = fetch_top_display_price(API_KEY, JAN)

    print("────────────────────────────")
    print(f"🔍 ヒット件数: {hit_count} 件")
    if tokens_used > 3:
        print("⚠️ トークンオーバー（3以上消費）")
    elif title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("❌ 価格取得失敗")
    print("────────────────────────────")
    print(f"🪙 消費トークン: {tokens_used}")
    print(f"💰 残トークン数: {tokens_left}")
    print("────────────────────────────")


❌ 商品情報が見つかりませんでした
────────────────────────────
🔍 ヒット件数: 0 件
❌ 価格取得失敗
────────────────────────────
🪙 消費トークン: 0
💰 残トークン数: 0
────────────────────────────


In [122]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5  # Amazon.co.jp
MAX_TOKENS_ALLOWED = 3  # 許容する最大トークン消費（3未満を許可、3以上は中断）

def fetch_top_display_price(api_key: str, code: str, max_retries=3) -> Tuple[Optional[str], Optional[int], int, int, int]:
    """
    ページトップに表示される価格（BuyBox）を最優先で取得。
    BuyBoxがなければ Prime か最初の出品を取得。
    tokensConsumed が MAX_TOKENS_ALLOWED 以上なら処理を中断して None を返す。
    戻り値: (title, price, hit_count, tokens_consumed, tokens_left)
    """
    url = (
        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"          # 履歴なしで軽量化
        "&offers=20"          # 出品情報を20件まで取得
        "&onlyLiveOffers=1"   # 現在出品のみ
        "&stats=0"            # BuyBox統計を含める
    )

    data = None
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()
            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(2)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0, 0, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        # data が None の可能性を考慮して安全に返す
        tokens_consumed = data.get("tokensConsumed", 0) if data else 0
        tokens_left = data.get("tokensLeft", 0) if data else 0
        return None, None, 0, tokens_consumed, tokens_left

    # トークン消費量チェック（ここで中断する）
    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)
    if tokens_consumed >= MAX_TOKENS_ALLOWED:
        print(f"❌ トークン消費が多すぎます: {tokens_consumed}（許容上限: {MAX_TOKENS_ALLOWED - 1}）")
        # トークン消費多すぎるので処理を行わず None を返す
        return None, None, len(data.get("products", [])), tokens_consumed, tokens_left

    hit_count = len(data["products"])

    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    # ————— ① BuyBox（トップ表示価格）を最優先 —————
    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count, tokens_consumed, tokens_left

    # ————— ② フォールバック：Prime 出品または最初の出品 —————
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count, tokens_consumed, tokens_left

    return None, None, hit_count, tokens_consumed, tokens_left


# -------------------------------------------
# 実行部
# -------------------------------------------
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "6971284200001"  # 例：商品JAN

    title, price, hit_count, tokens_used, tokens_left = fetch_top_display_price(API_KEY, JAN)

    print("────────────────────────────")
    print(f"🔍 ヒット件数: {hit_count} 件")
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("❌ 価格取得失敗")
    print("────────────────────────────")
    print(f"🪙 消費トークン: {tokens_used}")
    print(f"💰 残トークン数: {tokens_left}")
    print("────────────────────────────")


❌ トークン消費が多すぎます: 7（許容上限: 2）
────────────────────────────
🔍 ヒット件数: 3 件
❌ 価格取得失敗
────────────────────────────
🪙 消費トークン: 7
💰 残トークン数: 42
────────────────────────────


In [158]:
import requests

API_KEY =  "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"

url = f"https://api.keepa.com/token?key={API_KEY}"
res = requests.get(url)
print(res.json())


{'processingTimeInMs': 0, 'refillIn': 31688, 'refillRate': 5, 'timestamp': 1759733055449, 'tokenFlowReduction': 0.0, 'tokensConsumed': 0, 'tokensLeft': 34}


In [161]:
import requests
import time
from typing import Optional, Tuple

DOMAIN_JP = 5
MAX_TOKENS_ALLOWED = 10    # 既存のトークン閾値（併用する場合に有効）
MAX_SECONDS_ALLOWED = 10   # 許容する最大経過時間（秒）

def fetch_top_display_price(api_key: str, code: str, max_retries=3,
                            max_seconds: int = MAX_SECONDS_ALLOWED,
                            max_tokens: Optional[int] = MAX_TOKENS_ALLOWED
                           ) -> Tuple[Optional[str], Optional[int], int, int, int]:
    """
    ページトップに表示される価格を取得。呼び出し全体の最大許容秒数で中断可能。
    戻り値: (title, price, hit_count, tokens_consumed, tokens_left)
    """
    url = (
        # "https://api.keepa.com/product"
        # f"?key={api_key}"
        # f"&domain={DOMAIN_JP}"
        # f"&code={code}"
        # "&history=0"
        # "&offers=20"
        # "&onlyLiveOffers=0"
        # "&stats=0"


        "https://api.keepa.com/product"
        f"?key={api_key}"
        f"&domain={DOMAIN_JP}"
        f"&code={code}"
        "&history=0"
        "&offers=20"          # 出品情報は20件まで（維持）
        "&onlyLiveOffers=0"   # ✅ 再帰問い合わせ抑制
        "&buybox=1"           # ✅ BuyBox情報を直接取得
        "&stats=0"    
    )

    start = time.time()
    data = None
    for attempt in range(max_retries):
        # 経過時間チェック：リトライ前に時間切れなら中断
        elapsed = time.time() - start
        if max_seconds is not None and elapsed >= max_seconds:
            print(f"❌ 時間上限に到達しました: {elapsed:.2f}s（上限: {max_seconds}s）")
            tokens_consumed = data.get("tokensConsumed", 0) if data else 0
            tokens_left = data.get("tokensLeft", 0) if data else 0
            return None, None, 0, tokens_consumed, tokens_left

        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            data = resp.json()

            # 取得直後に経過時間チェック：レスポンス解析前に時間切れを確認
            elapsed = time.time() - start
            if max_seconds is not None and elapsed >= max_seconds:
                print(f"❌ 時間上限に到達しました（レスポンス取得後）: {elapsed:.2f}s")
                tokens_consumed = data.get("tokensConsumed", 0)
                tokens_left = data.get("tokensLeft", 0)
                return None, None, len(data.get("products", [])), tokens_consumed, tokens_left

            break
        except requests.exceptions.ReadTimeout:
            print(f"⚠️ タイムアウト（{attempt+1}/{max_retries}）: {code}")
            time.sleep(1)
        except Exception as e:
            print(f"⚠️ エラー: {e}")
            return None, None, 0, 0, 0

    if not data or "products" not in data or not data["products"]:
        print("❌ 商品情報が見つかりませんでした")
        tokens_consumed = data.get("tokensConsumed", 0) if data else 0
        tokens_left = data.get("tokensLeft", 0) if data else 0
        return None, None, 0, tokens_consumed, tokens_left

    tokens_consumed = data.get("tokensConsumed", 0)
    tokens_left = data.get("tokensLeft", 0)

    # オプションでトークン閾値も併用する
    if max_tokens is not None and tokens_consumed >= max_tokens:
        print(f"❌ トークン消費が多すぎます: {tokens_consumed}（許容上限: {max_tokens - 1}）")
        return None, None, len(data.get("products", [])), tokens_consumed, tokens_left

    hit_count = len(data["products"])
    product = data["products"][0]
    title = product.get("title", "タイトルなし")

    stats = product.get("stats") or {}
    for key in ("buyBoxShippingPrice", "buyBoxPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), hit_count, tokens_consumed, tokens_left

    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        p = chosen.get("price")
        ship = chosen.get("shipping")
        if isinstance(p, (int, float)) and p > 0:
            total = int(p) + (int(ship) if isinstance(ship, (int, float)) and ship > 0 else 0)
            return title, total, hit_count, tokens_consumed, tokens_left

    return None, None, hit_count, tokens_consumed, tokens_left


# 実行例は元のまま
if __name__ == "__main__":
    API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
    JAN = "4522654180816"
    title, price, hit_count, tokens_used, tokens_left = fetch_top_display_price(API_KEY, JAN)
    print("────────────────────────────")
    print(f"ヒット件数: {hit_count} 件")
    print("商品コード" + JAN)
    if title and price:
        print(f"商品タイトル: {title}")
        print(f"ページトップ表示価格: {price} 円")
    else:
        print("価格取得失敗")
    print("────────────────────────────")
    print(f"消費トークン: {tokens_used}")
    print(f"残トークン数: {tokens_left}")
    print("────────────────────────────")


────────────────────────────
ヒット件数: 1 件
商品コード4522654180816
商品タイトル: 森本産業 ﾋﾞｼﾞｭｰﾊﾞﾝｽｸﾘｯﾌﾟ ｻﾝﾘｵ ｸﾛﾐ RM-8436 H7.5×W8.2×D5.4cm
ページトップ表示価格: 1620 円
────────────────────────────
消費トークン: 5
残トークン数: 88
────────────────────────────


In [ ]:
chosen = ordered[0] if ordered else None